In [228]:
import numpy as np


import matplotlib.pyplot as plt
import random

## The data

The data is from electron microscopy - images of grpahene sheets. The typical image is $(256, 256)$ but we would like to take small patches of this and identify - is there a defect in the patch.

Lets go through a few of those things. Below we will look at 

* A full image
* A patch of an image with no defects
* A patch with a defect

### Full image

In [229]:
graphene = np.load('full-stack.npy') ## This has 180 different microgrpahs each (256, 256)
sample = np.squeeze(graphene[54])

In [230]:
print(sample.shape)

(256, 256)


In [231]:
plt.imshow(sample)
plt.colorbar();

#### Description
The blue (high electron density) correspnds to atoms and the green correspnds to background.

You can see that in the middle of the sheet most of it is a pretty regular array of hexagons, this is what we exepct for the perfect lattice. You will also hopefully see some spots where the hexagon is broken. This many of these are the result of missing atom defects. 

Notice the edges look a bit weird too. This is standard, and we will generally ignore the edges when analysing.



### Perfect patch

There is a training dataset of perfect patches 

In [232]:
pp = np.load('./perfect_patches.npy')
perfect_sample = pp[0] 
print(pp.shape)

(2279, 48, 48)


In [233]:
n1 = np.sum(perfect_sample, axis=0)
plt.plot(n1, label='Sum over vertical direction')
e1 = np.std(n1)/np.mean(n1)
n2 = np.sum(perfect_sample, axis=1)
e2 = np.std(n2)/np.mean(n2)
plt.plot(n2,label='Sum over horizontal direction')
e1, e2, np.mean(n1), np.mean(n2), np.std(n1), np.std(n2)

(0.12360269, 0.0072767125, 24.593985, 24.593985, 3.0398827, 0.17896335)

In [234]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

def plotVec(image,n1,n2,name):
    fig, axs = plt.subplots(2, 2,figsize=(16,16))
    axs[0, 0].imshow(image)
    axs[0, 0].set_title('Patch under inspection', fontsize=20)
    axs[0, 1].plot(n1,np.arange(1,n1.shape[0]+1),'r',label='H Direction')
    axs[0, 1].set_title('Horizontal projection', fontsize=20)
    axs[1, 0].plot(n2, 'b',label='V Direction')
    axs[1, 0].set_title('Vertical projection', fontsize=20)
    axs[1, 1].plot(n1, 'r', label='H projection')
    axs[1, 1].plot(n2, 'b', label='V projection')
    axs[1, 1].set_title('Both projections', fontsize=20)
    axs[0, 1].set(xlim=(20, 30))
    axs[1, 1].set(ylim=(20, 30))
    axs[1, 0].set(ylim=(20, 30))
    axs[1,1].legend(loc='best',fontsize=20)
    axs[0, 1].legend(loc='best',fontsize=20)
    axs[1, 0].legend(loc='best',fontsize=20)
    axs[0, 0].tick_params(axis='x', labelsize=16)
    axs[0, 0].tick_params(axis='y', labelsize=16)
    axs[0, 1].tick_params(axis='x', labelsize=16)
    axs[0, 1].tick_params(axis='y', labelsize=16)
    axs[1, 0].tick_params(axis='x', labelsize=16)
    axs[1, 0].tick_params(axis='y', labelsize=16)
    axs[1, 1].tick_params(axis='x', labelsize=16)
    axs[1, 1].tick_params(axis='y', labelsize=16)
    fig.savefig(name, format='pgf', bbox_inches='tight')
    



In [235]:
plotVec(perfect_sample,n1,n2,'perfect_sample.pgf')

In [236]:
plt.imshow(perfect_sample);
plt.show()

/tmp/ipykernel_20208/1837148048.py:2: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Defect patch

There is also a set of defective patches. Note - these are for testing only, not for training. I guess we could imagine training a classifier on these and the perfect patches - but that would be no fun :)

In [237]:
dp = np.load('./defect_patches.npy')
defect_sample = dp[22] 
print(dp.shape)


(32, 48, 48)


In [238]:
plt.figure(figsize=(5, 5))
plt.imshow(defect_sample);
plt.show()

/tmp/ipykernel_20208/1731622944.py:3: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [239]:
n1 = np.sum(defect_sample, axis=0)
plt.plot(n1)
e1 = np.std(n1)/np.mean(n1)
n2 = np.sum(defect_sample, axis=1)
e2 = np.std(n2)/np.mean(n2)
plt.plot(n2)
e1, e2, np.mean(n1), np.mean(n2), np.std(n1), np.std(n2)


(0.12030859, 0.03492741, 25.291075, 25.291077, 3.0427337, 0.8833518)

Get patches perfect and defect, and put them randomly in a list, with the ground truth saved

Have fun :-)

In [240]:
plotVec(defect_sample,n1,n2,'defect_sample.pgf')

In [241]:
def data(image):
    image0 = np.sum(image,axis=0)
    image1 = np.sum(image,axis=1)
    stdim0 = np.std(image0)
    stdim1 = np.std(image1)
    return [stdim0,stdim1]


In [242]:
ppL = []
dpL = []
for i in pp:
    ppL.append(data(i))
for i in dp:
    dpL.append(data(i))


In [243]:
patches = np.concatenate([pp, dp])
gt = np.concatenate([np.zeros(len(pp)), np.ones(len(dp))])
xy = list(zip(patches, gt)) # put labels with data in (data, label) tuple
random.shuffle(xy) # shuffle the tuples

In [244]:
patches, gt = list(zip(*xy)) # unzip the tuples to get patches and ground truths as vectors

In [245]:
len(patches)

2311